In [2]:
import pandas as pd
from faker import Faker
import random
import re
from typing import List, Dict
import string
from datetime import datetime, timedelta
import json




In [3]:
# Import FAKER Library in Canada Format

fake = Faker('en_CA')
fake_us= Faker('en_US')

In [ ]:
# Load the dataset

df = pd.read_csv('dataset_with_placeholder.csv')

In [4]:
# Generate PII

# SIN CA
def format_sin_with_dashes(sin_str):
    return f"{sin_str[:3]}-{sin_str[3:6]}-{sin_str[6:]}"

def luhn_algorithm(sin_without_check_digit):
    total = 0
    reverse_digits = sin_without_check_digit[::-1]
    
    for i, digit in enumerate(reverse_digits):
        num = int(digit)
        if i % 2 == 1:  
            num *= 2
            if num > 9:
                num -= 9
        total += num
    
    # ensure the total is a multiple of 10
    check_digit = (10 - (total % 10)) % 10
    return check_digit

def generate_canadian_sin():
    # generate a random 9-digit number

    # the first digit must be between 1 and 9
    sin_without_check_digit = ''.join([str(random.randint(1, 9)) if i == 0 else str(random.randint(0, 9)) for i in range(8)])
    
    # calculate the check digit
    check_digit = luhn_algorithm(sin_without_check_digit)
    
    # return the formatted SIN with dashes eg: 123-456-789
    sin_str = sin_without_check_digit + str(check_digit)
    return format_sin_with_dashes(sin_str)

# Canadian Passport

def generate_canadian_passport():
    """Generate a valid Canadian passport number (XX123456)."""
    letters = ''.join(random.choices(string.ascii_uppercase, k=2))  # Two uppercase letters
    digits = ''.join(random.choices(string.digits, k=6))  # Six digits
    return f"{letters}{digits}"


# Drivers License in Ontario

def generate_drivers_license():
    # Generate one random uppercase letter
    letter = random.choice(string.ascii_uppercase)

    # Generate 8 random digits
    digits_8 = ''.join(random.choices(string.digits, k=8))

    # Generate a two-digit year (must be at least 16 years before today)
    current_year = datetime.now().year
    min_year = current_year - 100  # Max possible range (99 years)
    max_year = current_year - 16  # At least 16 years old
    year = str(random.randint(min_year, max_year))[-2:]  # Take last two digits

    # Generate one digit that can be 0, 1, 5, or 6
    digit1 = random.choice(['0', '1', '5', '6'])

    # Generate one random digit
    digit2 = str(random.randint(0, 9))

    # Generate one digit that can be 0, 1, 2, or 3
    digit3 = random.choice(['0', '1', '2', '3'])

    # Generate one random digit
    digit4 = str(random.randint(0, 9))

    # Construct the license number
    license_number = f"{letter}{digits_8}{year}{digit1}{digit2}{digit3}{digit4}"
    
    return license_number


def add_dashes_to_dl():
    dl_number = generate_drivers_license()
    """Add a dash every 5 characters to a driver's license number."""
    # Split the driver's license number into chunks of 5 characters and join them with dashes
    return '-'.join([dl_number[i:i+5] for i in range(0, len(dl_number), 5)])

# Generate a random date in the past

def generate_date_yyyymmdd():
    date_obj = fake.date_object()  
    # RETURN "YYYYMMDD"
    date_str = date_obj.strftime('%Y%m%d')
    return date_str

# Add Space Credit Card Number
def generate_credit_card_number_spaced():
    """Generate a random credit card number with spaces every 4 digits."""
    cc_num = fake.credit_card_number()
    cc_num = re.sub(r'[\s-]', '', cc_num)
    spaced_cc_num = ' '.join(cc_num[i:i+4] for i in range(0, len(cc_num), 4))
    return spaced_cc_num

In [ ]:
# Placeholder for PII

PII_TYPES = {
    "[NAME]": fake.name,
    "[EMAIL]": fake.email,
    "[PHONE_NUMBER]": fake.phone_number,
    "[STREET_ADDRESS]": fake.street_address,
    "[PASSPORT_NUMBER]": generate_canadian_passport,
    "[DRIVER_LICENSE_NUMBER]": add_dashes_to_dl,
    "[POSTAL_CODE]": fake.postalcode,
    "[CREDIT_CARD_NUMBER]": fake.credit_card_number,
    "[DATE]": lambda: fake.date(pattern="%Y-%m-%d"),
    "[SSN]": generate_canadian_sin
}



# Remove the unrelated [CONTENT] from the text column
df['text'] = df['text'].str.replace(r'\[CONTENT\]', '', regex=True)
df['text'] = df['text'].str.strip(', ')  # remove leading/trailing commas

# Replace the placeholders with random PII values
def replace_placeholders(text):
    if not isinstance(text, str):  #  NaN
        return text, ""
    
    replaced_pii = []  # store the PII values that were replaced
    
    for placeholder, generator in PII_TYPES.items():
        if placeholder in text:
            value = generator() if callable(generator) else generator  # call the function if it's a function
            text = text.replace(placeholder, value, 1)  # only replace the first occurrence
            replaced_pii.append(value)  # store the replaced value
    
    return text, ", ".join(replaced_pii)

# Apply the function to the 'text' column
df[['text', 'pii_values']] = df['text'].apply(lambda x: pd.Series(replace_placeholders(x)))

# Extract the PII types from the 'pii_values' column
df['pii_types'] = df['pii_types'].str.replace(r'[\[\]]', '', regex=True).str.lower()
df

,text,pii_types,pii_values
0,"During the 38103399849486 audit, discrepancies...","'credit_card_number', 'passport_number'","OJ201940, 38103399849486"
1,If the 725 Daniel Haven provided does not matc...,"'street_address', 'postal_code'","725 Daniel Haven, R5X 9C9"
2,"Unless IK043144 is updated, andrea16@example.n...","'passport_number', 'email'","andrea16@example.net, IK043144"
3,"Unless TJ167911 is updated, 4405 William Radia...","'passport_number', 'street_address'","4405 William Radial, TJ167911"
4,"If 2001-01-12 doesn't match (704) 107-8764, pl...","'date', 'phone_number'","(704) 107-8764, 2001-01-12"
...,...,...,...
495,Could you confirm the 546-635-871 linked to Si...,"'ssn', 'name'","Sierra Humphrey, 546-635-871"
496,Has the 857-388-069 associated with X5Y4R5 bee...,"'ssn', 'postal_code'","X5Y4R5, 857-388-069"
497,Unless the 6011712438508479 is updated prior t...,"'credit_card_number', 'driver_license_number'","S1405-90543-90831, 6011712438508479"
498,Update 45788 Alicia Parkways and (552) 181-824...,"'street_address', 'phone_number'","(552) 181-8243, 45788 Alicia Parkways"


In [ ]:
def find_nth_occurrence(text, word, n):
    """Find the nth occurrence of word in text"""
    start = -1
    for _ in range(n):
        start = text.find(word, start + 1)
        if start == -1:
            return None  # If word not found n times
    end = start + len(word)
    return start, end

# Function to add 'pii_spans' column
def add_pii_spans(df):
    pii_spans = []
    
    for _, row in df.iterrows():
        text = row['text']
        pii_values = row['pii_values'].split(", ")  # Make sure pii_values is a list
        pii_types = row['pii_types'].split(", ")  # Make sure pii_types is a list
        
        # Ensure the lengths match
        if len(pii_values) != len(pii_types):
            print(f"⚠️ Warning: Length mismatch in row {row}")
            pii_spans.append([])
            continue
        
        # Count the occurrences of each word
        word_count = {word: 0 for word in pii_values}
        pii_span_list = []
        
        for idx, word in enumerate(pii_values):
            word_count[word] += 1
            nth_occurrence = word_count[word]
            
            # Find the nth occurrence of the word in the text
            span = find_nth_occurrence(text, word, nth_occurrence)
            if span:
                start, end = span
                pii_span_list.append({
                    'start': start,
                    'end': end,
                    'label': pii_types[idx]  # Use the corresponding PII type
                })
            else:
                pii_span_list.append({})  # If word not found, append an empty dict
        
        pii_spans.append(pii_span_list)
    
    df['pii_spans'] = pii_spans
    return df


df = add_pii_spans(df)

# Save the final output to a CSV file
df.to_csv("output_final.csv", index=False)

print("✅ PII spans saved to 'output_final.csv'")
df

✅ PII spans 计算完成，CSV 已保存！


,text,pii_types,pii_values,pii_spans
0,"During the 38103399849486 audit, discrepancies...","'credit_card_number', 'passport_number'","OJ201940, 38103399849486","[{'start': 50, 'end': 58, 'label': ''credit_ca..."
1,If the 725 Daniel Haven provided does not matc...,"'street_address', 'postal_code'","725 Daniel Haven, R5X 9C9","[{'start': 7, 'end': 23, 'label': ''street_add..."
2,"Unless IK043144 is updated, andrea16@example.n...","'passport_number', 'email'","andrea16@example.net, IK043144","[{'start': 28, 'end': 48, 'label': ''passport_..."
3,"Unless TJ167911 is updated, 4405 William Radia...","'passport_number', 'street_address'","4405 William Radial, TJ167911","[{'start': 28, 'end': 47, 'label': ''passport_..."
4,"If 2001-01-12 doesn't match (704) 107-8764, pl...","'date', 'phone_number'","(704) 107-8764, 2001-01-12","[{'start': 28, 'end': 42, 'label': ''date''}, ..."
...,...,...,...,...
495,Could you confirm the 546-635-871 linked to Si...,"'ssn', 'name'","Sierra Humphrey, 546-635-871","[{'start': 44, 'end': 59, 'label': ''ssn''}, {..."
496,Has the 857-388-069 associated with X5Y4R5 bee...,"'ssn', 'postal_code'","X5Y4R5, 857-388-069","[{'start': 36, 'end': 42, 'label': ''ssn''}, {..."
497,Unless the 6011712438508479 is updated prior t...,"'credit_card_number', 'driver_license_number'","S1405-90543-90831, 6011712438508479","[{'start': 84, 'end': 101, 'label': ''credit_c..."
498,Update 45788 Alicia Parkways and (552) 181-824...,"'street_address', 'phone_number'","(552) 181-8243, 45788 Alicia Parkways","[{'start': 33, 'end': 47, 'label': ''street_ad..."


In [ ]:

def find_nth_occurrence(text, word, n):
    """Find the nth occurrence of word in text"""
    start = -1
    for _ in range(n):
        start = text.find(word, start + 1)
        if start == -1:
            return None  # If word not found n times
    end = start + len(word)
    return start, end

# Function to add 'pii_spans' column in JSON format
def add_pii_spans(df):
    pii_spans = []
    
    for _, row in df.iterrows():
        text = row['text']
        pii_values = row['pii_values'].split(", ") if pd.notna(row['pii_values']) else []  #  NaN
        pii_types = row['pii_types'].split(", ") if pd.notna(row['pii_types']) else []  #  NaN
        
        # Ensure the lengths match
        if len(pii_values) != len(pii_types):
            print(f"⚠️ Warning: Length mismatch in row {row}")
            pii_spans.append([])
            continue
        
        # Count the occurrences of each word
        word_count = {word: 0 for word in pii_values}
        span_list = []
        
        for idx, word in enumerate(pii_values):
            word_count[word] += 1
            nth_occurrence = word_count[word]
            
            # Find the nth occurrence of the word in the text
            span = find_nth_occurrence(text, word, nth_occurrence)
            if span:
                start, end = span
                span_list.append({
                    "start": start,
                    "end": end,
                    "text": word,  
                    "labels": [pii_types[idx]] 
                })
        
        pii_spans.append(span_list)
    
    df['pii_spans'] = pii_spans

    # Encode the list of dicts as a JSON string
    df['pii_spans'] = df['pii_spans'].apply(json.dumps)

    return df


df = add_pii_spans(df)

# Save the final output to a CSV file
df.to_csv("output_final_jason_spans.csv", index=False)

print("✅ PII spans 计算完成，CSV 已保存！")


✅ PII spans 计算完成，CSV 已保存！


In [ ]:
from collections import Counter# 

pii_counter = Counter()

# Count the occurrences of each PII type
for pii_list in df['pii_types'].dropna():  #  NaN
    pii_items = pii_list.split(', ')  # Split the comma-separated values
    pii_counter.update(pii_items)

#  DataFrame
pii_count_df = pd.DataFrame(pii_counter.items(), columns=['pii_type', 'count'])

#  Sort the DataFrame by count in descending order
pii_count_df = pii_count_df.sort_values(by='count', ascending=False)
pii_count_df

,pii_type,count
8,'ssn',119
6,'phone_number',113
0,'credit_card_number',108
5,'date',106
2,'street_address',105
4,'email',98
3,'postal_code',92
7,'name',91
9,'driver_license_number',87
1,'passport_number',81
